In [1]:
import torch
from models import ConvAttnNet
from models import resnet
from models import convnext
from Datasets.datasets import MyDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import ops.trains as trains
import ops.tests as tests
# import ops.datasets as datasets
import ops.schedulers as schedulers

import models
import datetime
import os
import yaml

### Build Model

In [2]:
# torch.cuda.set_device(0)

In [2]:
# model = AlterNet.dnn_34(num_classes=1)
# model = resnet.dnn_50(num_classes=1)
model = convnext.convnext_small(num=1)

In [4]:
print(model)

AlterNet(
  (layer0): StemB(
    (layer0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (shortcut): Sequential()
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (sd): Identity()
    )
    (1): BasicBlock(
      (shortcut): Sequential()
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 

### Dataset

In [3]:
dataset_train = MyDataset('E:/datasets/project1/voxceleb_clip/train/trainset.csv')
dataset_test = MyDataset('E:/datasets/project1/voxceleb_clip/train/valset.csv')

In [4]:
dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=4, 
                           batch_size=64)
dataset_test = DataLoader(dataset_test, 
                          num_workers=4, 
                          batch_size=64)

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    2
))

Train: 36084, Test: 4010, Classes: 2


### Train

Define a TensorBoard writer:

In [5]:
dataset_name = 'ENF'

In [6]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = os.path.join("runs", dataset_name, model.name, current_time)
writer = SummaryWriter(log_dir)

# with open("%s/config.yaml" % log_dir, "w") as f:
#     yaml.dump(args, f)
with open("%s/model.log" % log_dir, "w") as f:
    f.write(repr(model))

print("Create TensorBoard log dir: ", log_dir)

Create TensorBoard log dir:  runs\ENF\convnext\20220408_112722


Train the model:

In [7]:
optim_args = {'name': 'SGD', 'lr': 1.0e-04, 'momentum': 0.9, 'nesterov': False, 'weight_decay': 0.0005, 
              'scheduler': {'name': 'MultiStepLR', 'milestones': [60, 120, 160], 'gamma': 0.2}}
train_args = {'warmup_epochs': 1, 'epochs': 200, 'batch_size': 64, 'binary': True}
val_args = {'batch_size': 128, 'n_ff': 1}

In [8]:
gpu = torch.cuda.is_available()
optimizer, train_scheduler = trains.get_optimizer(model, **optim_args)
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(dataset_train) * train_args.get("warmup_epochs", 0))

In [ ]:
trains.train(model, optimizer,
             dataset_train, dataset_test,
             train_scheduler, warmup_scheduler,
             train_args, val_args, gpu,
             writer, 
             snapshot=2, dataset_name=dataset_name, uid=current_time)  # Set `snapshot=N` to save snapshots every N epochs.

(261.66 sec/epoch) Warmup epoch: 0, Loss: 0.3586, lr: 1.000e-04
NLL: 0.0000e+00, Cutoffs: 0.0 %, 90.0 %, Accs: 100.000 %, 100.000 %, Uncs: 0.000 %, 0.000 %, IoUs: 100.000 %, 100.000 %, Freqs: 100.000 %, 100.000 %, Top-5: 100.000 %, Brier: 0.000, ECE: 100.000 %, ECE±: -100.000 %
The model is warmed up: 296.01 sec 

(258.16 sec/epoch) Epoch: 0, Loss: 0.3564, lr: 1.000e-04
NLL: 0.0000e+00, Cutoffs: 0.0 %, 90.0 %, Accs: 100.000 %, 100.000 %, Uncs: 0.000 %, 0.000 %, IoUs: 100.000 %, 100.000 %, Freqs: 100.000 %, 100.000 %, Top-5: 100.000 %, Brier: 0.000, ECE: 100.000 %, ECE±: -100.000 %


save the model

In [10]:
models.save(model, dataset_name, current_time, optimizer=optimizer)

### Load weights & Evaluate

In [9]:
models.load_snapshot(model, 'ENF', '20220406_205617', 3)

In [11]:
models.load_snapshot(model, dataset_name, '20220406_143309', 11, optimizer=optimizer)

In [12]:
models.load_snapshot(model, dataset_name, '20220324_113036', 11, optimizer=optimizer)

In [7]:
models.load_snapshot(model, dataset_name, '20220324_173157', 5, optimizer=optimizer)

create evaluate set

In [8]:
M1_eval_set = MyDataset('D:/projet_1(audio_classify)/project_5/M1/2s/trainset.csv')
M2_eval_set = MyDataset('D:/projet_1(audio_classify)/project_5/M2/2s/trainset.csv')

In [9]:
dataset_M1 = DataLoader(M1_eval_set, 
                           shuffle=True, 
                           num_workers=2, 
                           batch_size=128)
dataset_M2 = DataLoader(M2_eval_set, 
                          num_workers=2, 
                          batch_size=128)

print("M1: %s, M2: %s, Classes: %s" % (
    len(dataset_M1.dataset), 
    len(dataset_M2.dataset), 
    2
))

M1: 2002, M2: 3002, Classes: 2


evaluate

In [10]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1, 2, 3, 4, 5, 10, 20, 50]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_M1, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s.csv" % (dataset_name, model.name, current_time))
tests.save_metrics(metrics_dir, metrics_list)

N: 1, NLL: 0.6475, Cutoffs: 0.0 %, 90.0 %, Accs: 65.285 %, 89.370 %, Uncs: 0.000 %, 83.741 %, IoUs: 44.773 %, 44.685 %, Freqs: 100.000 %, 53.097 %, Top-5: 100.000 %, Brier: 0.458, ECE: 18.762 %, ECE±: 18.762 %
N: 2, NLL: 0.6449, Cutoffs: 0.0 %, 90.0 %, Accs: 64.486 %, 89.558 %, Uncs: 0.000 %, 84.388 %, IoUs: 43.697 %, 44.779 %, Freqs: 100.000 %, 53.097 %, Top-5: 100.000 %, Brier: 0.459, ECE: 19.576 %, ECE±: 19.576 %
N: 3, NLL: 0.6444, Cutoffs: 0.0 %, 90.0 %, Accs: 64.086 %, 88.931 %, Uncs: 0.000 %, 83.588 %, IoUs: 43.161 %, 44.465 %, Freqs: 100.000 %, 53.247 %, Top-5: 100.000 %, Brier: 0.458, ECE: 19.865 %, ECE±: 19.865 %
N: 4, NLL: 0.6462, Cutoffs: 0.0 %, 90.0 %, Accs: 64.885 %, 88.889 %, Uncs: 0.000 %, 83.215 %, IoUs: 44.234 %, 44.444 %, Freqs: 100.000 %, 53.047 %, Top-5: 100.000 %, Brier: 0.458, ECE: 19.131 %, ECE±: 19.131 %
N: 5, NLL: 0.6484, Cutoffs: 0.0 %, 90.0 %, Accs: 64.635 %, 88.649 %, Uncs: 0.000 %, 82.910 %, IoUs: 43.898 %, 44.325 %, Freqs: 100.000 %, 53.247 %, Top-5: 100.0

KeyboardInterrupt: 